In [1]:
import gcp

gcp.get_compute_client()

In [10]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/joe/Dev/key/dauntless-graph-293915-0a54f42271b4.json"

In [11]:
def implicit():
    from google.cloud import storage

    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    storage_client = storage.Client()

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)

In [12]:
implicit()

[]
